<h1>Team Log Grabber</h1>
<p>This notebook grabs the regular season logs for every hockey team from 1990 on</p>

In [1]:
import pandas as pd
import numpy as np
import requests as r
import re
from bs4 import BeautifulSoup as bs

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

<h2>First Grab a list of teams</h2>
<p>All the hockey teams are linked via a link /teams/XXX/history.html.  Defunct teams (or teams like the Atlanta Thrashers are listed as the team to which they transfered to.</p>

In [19]:
#first get a link to their histories

link_list = [] #list of links to the respective teams
url = 'https://www.hockey-reference.com'
t = r.get(url+'/teams')
soup = bs(t.text, 'html.parser')
links = soup.find_all('a')
for l in links:
    if 'history.html' in l['href']:
        link_list.append(l['href'])

In [81]:
#now create a dataframe consisting of the season, the team name, team abbreviation and the link season_log
teamDataFrame = pd.DataFrame(columns=['Season','team_name','team_abbr','link'])

for link in link_list:
    page = r.get(url+link)
    soup = bs(page.text, 'html.parser')
    links = soup.find_all('table')
    for row in links[0].find_all('tr')[1:]:

        #get season & link
        main_c = row.find_all('th')[0]
        link = main_c.find_all('a')

        if len(link)>0:
            link = link[0]['href'] #this gets the link to team main page

        if main_c['data-stat']=='season':
            season = main_c.text #this gets the season
        for c in row.find_all('td'):
            if c['data-stat']=='team_name':
                tname = c.text.strip('*')   #here's the team name
        abbr = re.search(r'teams\/(\w{3})\/',link)[1]  #the abbreviation is extracted from the link
        team_gamelog = 'teams/'+abbr+'/'+str(int(season[:4])+1)+'_gamelog.html'

        #record if the season is after 1990
        if int(season[:4])<1990:   #we don't care before 1990
            break
        else:
            teamDataFrame = teamDataFrame.append(dict(zip(teamDataFrame.columns,[season,tname,abbr,team_gamelog])),ignore_index=True)

teamDataFrame
teamDataFrame.head()

,Season,team_name,team_abbr,link
0,2019-20,Anaheim Ducks,ANA,teams/ANA/2020_gamelog.html
1,2018-19,Anaheim Ducks,ANA,teams/ANA/2019_gamelog.html
2,2017-18,Anaheim Ducks,ANA,teams/ANA/2018_gamelog.html
3,2016-17,Anaheim Ducks,ANA,teams/ANA/2017_gamelog.html
4,2015-16,Anaheim Ducks,ANA,teams/ANA/2016_gamelog.html
...,...,...,...,...
820,2003-04,Atlanta Thrashers,ATL,teams/ATL/2004_gamelog.html
821,2002-03,Atlanta Thrashers,ATL,teams/ATL/2003_gamelog.html
822,2001-02,Atlanta Thrashers,ATL,teams/ATL/2002_gamelog.html
823,2000-01,Atlanta Thrashers,ATL,teams/ATL/2001_gamelog.html


In [84]:
teamDataFrame['team_abbr'].value_counts()

NYR    29
DET    29
EDM    29
STL    29
CHI    29
PHI    29
PIT    29
MTL    29
NYI    29
TOR    29
VAN    29
WSH    29
BOS    29
LAK    29
NJD    29
CGY    29
BUF    29
SJS    28
OTT    27
TBL    27
DAL    26
FLA    26
COL    24
CAR    22
NSH    21
MIN    19
CBJ    19
PHX    17
ANA    14
MDA    12
ATL    11
WPG     9
HAR     7
WIN     6
ARI     6
QUE     5
VEG     3
MNS     3
Name: team_abbr, dtype: int64